In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from torch.utils.data import TensorDataset, DataLoader, Subset
import random

In [ ]:
data = np.load('/home/snu/Downloads/breastmnist_224.npz', allow_pickle=True)

all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # grayscale → 3-channel
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

images = torch.stack([transform(img) for img in all_images])
labels = torch.tensor(all_labels).long()

In [ ]:
dataset = TensorDataset(images, labels)
class0_indices = [i for i in range(len(labels)) if labels[i] == 0]
class1_indices = [i for i in range(len(labels)) if labels[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_indices, min(1000, len(class0_indices)))
sampled_class1 = random.sample(class1_indices, min(1000, len(class1_indices)))
combined_indices = sampled_class0 + sampled_class1
random.shuffle(combined_indices)
print()
final_dataset = Subset(dataset, combined_indices)
final_loader = DataLoader(final_dataset, batch_size=64, shuffle=False)

print(f"Number of samples in class 0: {len(sampled_class0)}")
print(f"Number of samples in class 1: {len(sampled_class1)}")
device = "cuda" if torch.cuda.is_available() else "cpu"


Number of samples in class 0: 210
Number of samples in class 1: 570


In [ ]:
vit = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')  # DINO ViT-B/16
vit.eval().to(device)

vit_feats = []
y_list = []

with torch.no_grad():
    for imgs, lbls in final_loader:
        imgs = imgs.to(device)
        feats = vit(imgs)  # (N, 768)
        vit_feats.append(feats.cpu())
        y_list.extend(lbls.cpu().tolist())

F = torch.cat(vit_feats, dim=0).numpy().astype(np.float32)
y_labels = np.array(y_list).astype(np.int64)

print("Feature shape:", F.shape)
print("Label shape:", y_labels.shape)

Using cache found in /home/snu/.cache/torch/hub/facebookresearch_dino_main


Feature shape: (780, 768)
Label shape: (780,)


In [ ]:
kmeans = KMeans(n_clusters=2, random_state=11, max_iter=5000)
kmeans.fit(F)

klabels_trans = kmeans.transform(F)
klabels_trans = klabels_trans / (klabels_trans.sum(axis=1, keepdims=True) + 1e-10)

y_pred = np.argmin(klabels_trans, axis=1)

In [ ]:
acc = accuracy_score(y_labels, y_pred)
inv_acc = accuracy_score(y_labels, 1 - y_pred)
if inv_acc > acc:
    acc = inv_acc
    y_pred = 1 - y_pred
    klabels_trans[:, [0, 1]] = klabels_trans[:, [1, 0]]  # swap probs

y_pred_proba = klabels_trans[:, 1]

prec = precision_score(y_labels, y_pred, zero_division=0)
rec = recall_score(y_labels, y_pred, zero_division=0)
f1 = f1_score(y_labels, y_pred, zero_division=0)
logloss = log_loss(y_labels, y_pred_proba)

print("===== KMeans Results (PneumoniaMNIST) =====")
print("Accuracy Score :", acc)
print("Precision Score:", prec)
print("Recall Score   :", rec)
print("F1 Score       :", f1)
print("Log Loss       :", logloss)

===== KMeans Results (PneumoniaMNIST) =====
Accuracy Score : 0.55
Precision Score: 0.6987295825771325
Recall Score   : 0.6754385964912281
F1 Score       : 0.6868867082961642
Log Loss       : 0.7027153150086075


In [ ]:
print(y_pred[:20])

[0 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1]


In [ ]:
print(y_labels[:20])

[1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0]


In [ ]:
num_runs = 10

acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

for run in range(num_runs):
    print(f"\n--- Run {run+1}/{num_runs} ---")
    np.random.seed(run)
    torch.manual_seed(run)

    kmeans = KMeans(n_clusters=2, random_state=run, max_iter=5000)
    kmeans.fit(F)

    klabels_trans = kmeans.transform(F)
    klabels_trans = klabels_trans / (klabels_trans.sum(axis=1, keepdims=True) + 1e-10)

    y_pred = np.argmin(klabels_trans, axis=1)

    acc = accuracy_score(y_labels, y_pred)
    inv_acc = accuracy_score(y_labels, 1 - y_pred)
    if inv_acc > acc:
        acc = inv_acc
        y_pred = 1 - y_pred
        klabels_trans[:, [0, 1]] = klabels_trans[:, [1, 0]]

    y_pred_proba = klabels_trans[:, 1]

    prec = precision_score(y_labels, y_pred, zero_division=0)
    rec = recall_score(y_labels, y_pred, zero_division=0)
    f1 = f1_score(y_labels, y_pred, zero_division=0)
    logloss = log_loss(y_labels, y_pred_proba)

    acc_scores.append(acc)
    prec_scores.append(prec)
    rec_scores.append(rec)
    f1_scores.append(f1)
    log_losses.append(logloss)

    print(f"Run {run+1} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | "
          f"F1: {f1:.4f} | LogLoss: {logloss:.4f}")

print("\n================ FINAL SUMMARY ================\n")
print(f"{'Metric':>15} | {'Mean':>10} ± {'Std':<10}")
print("-" * 50)
print(f"{'Accuracy':>15} | {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"{'Precision':>15} | {np.mean(prec_scores):.4f} ± {np.std(prec_scores):.4f}")
print(f"{'Recall':>15} | {np.mean(rec_scores):.4f} ± {np.std(rec_scores):.4f}")
print(f"{'F1 Score':>15} | {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"{'Log Loss':>15} | {np.mean(log_losses):.4f} ± {np.std(log_losses):.4f}")


--- Run 1/10 ---
Run 1 | Acc: 0.5346 | Prec: 0.6920 | Rec: 0.6544 | F1: 0.6727 | LogLoss: 0.7009

--- Run 2/10 ---
Run 2 | Acc: 0.5013 | Prec: 0.8987 | Rec: 0.3579 | F1: 0.5119 | LogLoss: 0.6942

--- Run 3/10 ---
Run 3 | Acc: 0.5705 | Prec: 0.7058 | Rec: 0.7070 | F1: 0.7064 | LogLoss: 0.7056

--- Run 4/10 ---
Run 4 | Acc: 0.5346 | Prec: 0.6927 | Rec: 0.6526 | F1: 0.6721 | LogLoss: 0.7006

--- Run 5/10 ---
Run 5 | Acc: 0.5436 | Prec: 0.6960 | Rec: 0.6667 | F1: 0.6810 | LogLoss: 0.7015

--- Run 6/10 ---
Run 6 | Acc: 0.5346 | Prec: 0.6920 | Rec: 0.6544 | F1: 0.6727 | LogLoss: 0.7009

--- Run 7/10 ---
Run 7 | Acc: 0.5449 | Prec: 0.8413 | Rec: 0.4649 | F1: 0.5989 | LogLoss: 0.6980

--- Run 8/10 ---
Run 8 | Acc: 0.5346 | Prec: 0.6920 | Rec: 0.6544 | F1: 0.6727 | LogLoss: 0.7009

--- Run 9/10 ---
Run 9 | Acc: 0.5346 | Prec: 0.6920 | Rec: 0.6544 | F1: 0.6727 | LogLoss: 0.7009

--- Run 10/10 ---
Run 10 | Acc: 0.5769 | Prec: 0.7069 | Rec: 0.7193 | F1: 0.7130 | LogLoss: 0.7075

================ 